In [1]:
import pandas as pd

In [2]:
cell_tidy_data = pd.read_msgpack(
    '/home/hanliu/project/mouse_rostral_brain/study/ClusteringSummary/Summary/TotalClusteringResults.msg'
)

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The read_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
mc_study_design = {
    'PT': ['PT-L5'],
    'Other-Cortex-Exc': ['NP-L6', 'CT-L6', 'L6b'],
    'OLF-Exc': ['OLF-Exc', 'CLA', 'EP'],
    'IT': ['IT-L5', 'IT-L6', 'IT-L23', 'IT-L4'],
    'HPF-Exc': ['CA3', 'CA1', 'CA3-St18', 'Gfra1', 'IG-CA2', 'DG-po', 'DG'],
    'MGECGE': ['MGE-Sst', 'Unc5c', 'CGE-Lamp5', 'CGE-Vip', 'MGE-Pvalb'],
    'Other-Inh': [
        'PAL-Inh', 'MSN-D2', 'OLF', 'Foxp2', 'MSN-D1', 'LSX-Inh', 'D1L-Fstl4',
        'D1L-PAL', 'Chd7'
    ],
    'NonN':
    ['ODC', 'PC', 'ANP', 'OPC', 'ASC', 'MGC', 'VLMC', 'EC', 'VLMC-Pia']
}

In [4]:
cluster_map = {}
for group_name, major_types in mc_study_design.items():
    for sub_type in cell_tidy_data[cell_tidy_data['MajorType'].isin(major_types)]['SubType'].unique():
        if 'Outlier' in sub_type:
            continue
        else:
            cluster_map[sub_type.replace(' ', '_')] = group_name
subtype_group = pd.Series(cluster_map)
subtype_group.to_csv('SubTypeGroup.csv', header=False)

In [5]:
with pd.HDFStore('/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/DMRInfo.h5') as hdf:
    print(hdf.keys())
    dmr_rate = hdf['Rate']
    dmr_bed = hdf['bed']
dmr_rate.shape

['/Rate', '/bed']


(4722053, 161)

In [6]:
for group, subtypes in subtype_group.groupby(subtype_group):
    this_rate = dmr_rate.loc[:, subtypes.index]
    dmr_delta = this_rate.max(axis=1) - this_rate.min(axis=1)
    dmr_mean = this_rate.mean(axis=1)
    # either global hypo, or delta is large
    judge = (dmr_delta > 0.3) | (dmr_mean < 0.4)

    this_bed = dmr_bed.loc[judge, :]
    this_bed.reset_index().iloc[:, [1, 2, 3, 0]].to_csv(f'group_dmr/{group}.DMRs.bed',
                                                        index=None,
                                                        header=None,
                                                        sep='\t')
    print(group, this_bed.shape[0], len(subtypes))

HPF-Exc (2831785, 4)
IT (1583131, 4)
MGECGE (2479056, 4)
NonN (3578887, 4)
OLF-Exc (2024678, 4)
Other-Cortex-Exc (1648933, 4)
Other-Inh (3117380, 4)
PT (1602220, 4)


In [10]:
cell_tidy_data['StudyGroup'] = cell_tidy_data['SubType'].apply(lambda i: subtype_group.get(i.replace(' ', '_'), ''))
for group, sub_tidy_data in cell_tidy_data.groupby('StudyGroup'):
    if group != '':
        sub_tidy_data.to_msgpack(f'group_dmr/{group}.cells.msg')